In [ ]:
# Instalação das dependências necessárias
!pip install "tensorflow[and-cuda]" tensorflow-hub
!pip install -q tensorflow tensorflow_hub
!pip install -q scikit-learn pandas numpy tqdm

# Para usar o BGE-M3
!pip install sentence-transformers

In [ ]:
# Verificação da instalação do TensorFlow com suporte a GPU
import tensorflow as tf, pprint
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))
pprint.pprint(tf.sysconfig.get_build_info())  # mostra cuda_version e cudnn_version

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tqdm import tqdm

#Para usar o BGE-M3
from sentence_transformers import SentenceTransformer 

2025-09-04 23:28:02.698616: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ricardo/Mestrado/TAIA/NLP-Embedding/.venv/lib/python3.10/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
/home/ricardo/Mestrado/TAIA/NLP-Embedding/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Carregando os dados
df = pd.read_csv('eventos_artigos_filtrados.csv');
df.shape
#df.head()

(12629, 4)

In [3]:
# Preparação dos dados
df = df.drop(columns=['evento_nome', 'resumo'])
df = df.dropna(subset=['titulo_artigo', 'ano_edicao'])
df['antes_2023'] = df['ano_edicao'].apply(lambda x: 1 if x < 2023 else 0)

df["titulo_artigo"] = df["titulo_artigo"].astype(str).str.strip()
df["antes_2023"] = df["antes_2023"].astype(int).values

#df.head()

In [ ]:
#Incluir aqui outros tratamentos, SFC, normalização, etc.

# Teste com BGE-M3

In [4]:
titulos = df['titulo_artigo'].tolist()
labels = df['antes_2023'].values

# Carregamento do Modelo BGE-M3 ---
model = SentenceTransformer("BAAI/bge-m3")
print("BGE-M3 carregado com sucesso.")

# Execução da Geração de Embeddings ---
embeddings_gerados = model.encode(
    titulos,
    batch_size=32,
    show_progress_bar=True
)

print(f"\nEmbeddings gerados com sucesso! Shape final: {embeddings_gerados.shape}")

# --- Salvamento dos Resultados ---
np.save('embeddings_titulos.npy', embeddings_gerados)
np.save('labels.npy', labels)

BGE-M3 carregado com sucesso.


Batches: 100%|██████████| 395/395 [00:40<00:00,  9.87it/s]



Embeddings gerados com sucesso! Shape final: (12629, 1024)


# CLASSIFICADOR 

In [5]:
# Carregamento dos Dados Pré-processados
# Usou 10Gb VRAM

X = np.load('embeddings_titulos.npy')
y = np.load('labels.npy')

print(f"Dados carregados. Shape de X: {X.shape}, Shape de y: {y.shape}")

# --- Divisão em Treino e Teste ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.\n")


# --- Construção do Modelo de Classificação ---
model = tf.keras.Sequential([
    # A camada de entrada espera vetores com 1024 dimensões (o tamanho do embedding ELMo)
    tf.keras.layers.InputLayer(input_shape=(1024,)),
    
    # Camada oculta para aprender padrões nos embeddings
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5), # Dropout para evitar overfitting
    
    # Camada de saída para classificação binária
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# --- Compilação do Modelo ---
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Treinamento do Modelo ---
print("\nIniciando o treinamento do classificador...")
history = model.fit(
    X_train,
    y_train,
    epochs=45,
    batch_size=4,
    validation_data=(X_test, y_test),
    verbose=1
)

# --- Avaliação Final ---
print("\nAvaliação final no conjunto de teste:")
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia final no teste: {accuracy:.4f}')

Dados carregados. Shape de X: (12629, 1024), Shape de y: (12629,)
Dados divididos: 10103 para treino, 2526 para teste.



/home/ricardo/Mestrado/TAIA/NLP-Embedding/.venv/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1757042661.655359  227939 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7485 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:0a:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,329 (513.00 KB)

...


Iniciando o treinamento do classificador...
Epoch 1/45


2025-09-04 23:24:22.610427: I external/local_xla/xla/service/service.cc:163] XLA service 0x7ff82c0069b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-04 23:24:22.610440: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-09-04 23:24:22.623237: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-04 23:24:22.700306: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


 158/2526 ━━━━━━━━━━━━━━━━━━━━ 2s 961us/step - accuracy: 0.7643 - loss: 0.5852

I0000 00:00:1757042663.302290  231400 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2526/2526 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7275 - loss: 0.5459 - val_accuracy: 0.7328 - val_loss: 0.5294
Epoch 2/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7356 - loss: 0.5179 - val_accuracy: 0.7379 - val_loss: 0.5249
Epoch 3/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7420 - loss: 0.5060 - val_accuracy: 0.7399 - val_loss: 0.5190
Epoch 4/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7495 - loss: 0.5006 - val_accuracy: 0.7411 - val_loss: 0.5221
Epoch 5/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7602 - loss: 0.4882 - val_accuracy: 0.7439 - val_loss: 0.5251
Epoch 6/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7629 - loss: 0.4803 - val_accuracy: 0.7502 - val_loss: 0.5185
Epoch 7/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7725 - loss: 0.4697 - val_accuracy: 0.7387 - val_loss: 0.5177
Epoch 8/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7792 - loss: 0.4538 - val_accurac

In [6]:
model.save('meu_classificador_com_BGE-M3.keras')

# TESTE COM DADOS DIFERENTES DO DATASET A PARIR DE UM MODELO GERADO.

In [2]:
# 1. Carregar o Modelo de Embedding (BGE-M3)
embedding_model = SentenceTransformer("BAAI/bge-m3")

# 2. Carregar o seu Classificador Keras treinado
classificador = tf.keras.models.load_model('meu_classificador_com_BGE-M3.keras')

novos_titulos = [
    "Um estudo sobre o impacto da IA generativa no desenvolvimento de software",
    "Implementação de algoritmos de parsing para compiladores em Pascal",
    "Otimização de redes neurais para sistemas embarcados de baixa potência",
    "Análise de sistemas de informação com metodologia estruturada",
    "Fine-tuning de LLMs para tarefas de tradução em português",
    "Knowledge graphs para recomendação de produtos em e-commerce",
    "Análise de sentimentos em redes sociais utilizando BERT",
    "Desenvolvimento de chatbots inteligentes com GPT-3",
    "Avaliação de técnicas de segurança em redes de computadores",
    "Estudo sobre algoritmos de compressão de dados para grandes volumes",
    "Aplicações de aprendizado por reforço em jogos digitais",
    "Técnicas de visualização de dados para análise exploratória",
    "Desenvolvimento de sistemas distribuídos com microserviços",
    "Análise de desempenho de bancos de dados NoSQL em aplicações web",
    "Estudo sobre algoritmos de criptografia para segurança de dados",
    "Implementação de redes neurais convolucionais para reconhecimento de imagens",
    "Desenvolvimento de aplicações móveis com Flutter",
    "Técnicas de mineração de dados para detecção de fraudes financeiras",
    "Estudo sobre a evolução dos sistemas operacionais ao longo das décadas",
    "Fine-tuning de LLMs para tarefas de tradução em português",
    "knowledge distillation em modelos de linguagem grande",
    "Compact models for edge AI applications",
    "Analysis of Criminal Patterns in Police Report Narratives using Spectral Clustering with K-means",
    "Knowledge Distillation in Compact Models: An Approach Applied to Text Processing for Public Security",
    "Inteligência Artificial e sociedade: avanços e riscos"
]

novos_embeddings = embedding_model.encode(novos_titulos)
predicoes_prob = classificador.predict(novos_embeddings)

# Resultados
for i, titulo in enumerate(novos_titulos):
    prob = predicoes_prob[i][0]
    
    if prob > 0.5:
        # Classe 1
        classificacao = "ANTES de 2023"
        confianca = prob * 100
    else:
        # Classe 0
        classificacao = "2023 ou DEPOIS"
        confianca = (1 - prob) * 100
        
    print(f"Título: '{titulo}'")
    print(f"  -> Classificação: {classificacao} (Confiança: {confianca:.2f}%)\n")

I0000 00:00:1757042896.126251  237860 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7631 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:0a:00.0, compute capability: 8.6
2025-09-04 23:28:16.964021: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f401c003bf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-04 23:28:16.964033: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-09-04 23:28:16.968619: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-04 23:28:16.980958: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step
Título: 'Um estudo sobre o impacto da IA generativa no desenvolvimento de software'
  -> Classificação: 2023 ou DEPOIS (Confiança: 96.48%)

Título: 'Implementação de algoritmos de parsing para compiladores em Pascal'
  -> Classificação: ANTES de 2023 (Confiança: 99.71%)

Título: 'Otimização de redes neurais para sistemas embarcados de baixa potência'
  -> Classificação: ANTES de 2023 (Confiança: 98.81%)

Título: 'Análise de sistemas de informação com metodologia estruturada'
  -> Classificação: ANTES de 2023 (Confiança: 100.00%)

Título: 'Fine-tuning de LLMs para tarefas de tradução em português'
  -> Classificação: 2023 ou DEPOIS (Confiança: 92.34%)

Título: 'Knowledge graphs para recomendação de produtos em e-commerce'
  -> Classificação: ANTES de 2023 (Confiança: 80.73%)

Título: 'Análise de sentimentos em redes sociais utilizando BERT'
  -> Classificação: ANTES de 2023 (Confiança: 100.00%)

Título: 'Desenvolvimento de chatbots inteligentes com

I0000 00:00:1757042897.306642  238200 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
